In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
from glob import glob

files = sorted(glob('*-topics.json'))
files[0], len(files)

('AnimeShrine-topics.json', 59)

In [3]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_text(url):
    texts = []
    try:
        r_ = requests.get(url)
        soup = BeautifulSoup(r_.content, "lxml")
        divs = soup.find_all('div', {'class': 'post_text'})
        for div in divs:
            t = BeautifulSoup(str(div).replace('<br/>', '\n')).text.strip()
            if len(t):
                texts.append(t)
    except Exception as e:
        pass
    return texts

In [4]:
limit = 20
worker_size = 1
per_batch = worker_size * limit

for k in range(len(files)):
    f = files[k]
    print(k, f)
    directory = f.split('-topics')[0]
    os.makedirs(f'done-{directory}', exist_ok = True)
    os.makedirs(directory, exist_ok = True)
    
    with open(f) as fopen:
        all_a = json.load(fopen)
    
    for topic in tqdm(sorted(all_a)):
    
        base_dir = topic[1:].replace('/', '-')
        os.makedirs(os.path.join(directory, base_dir), exist_ok = True)
        os.makedirs(f'done-{directory}', exist_ok = True)

        base_dir_filename = os.path.join(f'done-{directory}', f'{base_dir}.json')

        if os.path.exists(base_dir_filename):
            continue

        need_break = False
        last_texts = []
        for i in range(0, 100000, per_batch):
            filename = os.path.join(directory, base_dir, f'{i}.json')
            if os.path.exists(filename):
                continue
            aranged = np.arange(i, i + per_batch, limit)
            urls = []
            for a_ in aranged:
                if a_ == 0:
                    a_ = ''
                else:
                    a_ = f'/+{a_}'
                url = f'https://forum.lowyat.net/{topic}{a_}'
                urls.append(url)

            texts = []

            with ThreadPoolExecutor(max_workers=worker_size) as executor:
                futures = {executor.submit(get_text, item): item for item in urls}

                for future in as_completed(futures):
                    r = future.result()
                    texts.extend(r)
                    if not len(r):
                        need_break = True

                    s = set(texts)
                    if len(set(last_texts) & s) == len(s):
                        need_break = True

                    last_texts = texts[:]

            with open(filename, 'w') as fopen:
                json.dump(texts, fopen)

            time.sleep(random.uniform(0.5, 1.0))

            if need_break:
                break

        with open(base_dir_filename, 'w') as fopen:
            json.dump({'status': True}, fopen) 

0 AnimeShrine-topics.json


100%|████████████████████████████████████| 3672/3672 [00:00<00:00, 44358.47it/s]


1 Announcements-topics.json


100%|██████████████████████████████████████| 156/156 [00:00<00:00, 53684.89it/s]


2 AppleByte-topics.json


...

100%|██████████████████████████████████████| 199/199 [00:00<00:00, 50112.06it/s]


57 Travel&Living-topics.json


100%|████████████████████████████████████| 3847/3847 [00:00<00:00, 75271.09it/s]


58 Windows11-topics.json


100%|████████████████████████████████████████| 40/40 [00:00<00:00, 49475.72it/s]
